In [11]:
import pandas as pd
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from ddi.utils import df_optimized, get_data_filepath

In [36]:
def get_data():
    '''retrieve and clean the final_dataset'''
    df = pd.read_csv(get_data_filepath('final_dataset.csv'))
    df.drop(columns =[col for col in df.columns if 'Unnamed' in col], inplace = True)
    df.drop(columns = ['86'], inplace = True) 
    df = df_optimized(df)
    return df

def preprocess(df):
    '''transform the df into pca features'''
    X = df[df.columns[89:]]  # check with marcus what are X and y columns -
    y = df[df.columns[3:89]]
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3 )

    '''
    creatin a pipeline to process the data
    NOTE: 80% of the variation can be explained by 46 principal components from param_search.py
    '''
    preproc = make_pipeline(StandardScaler(),PCA(n_components =46))
    preproc.fit(X_train)
    X_train = preproc.transform(X_train)
    X_test = preproc.transform(X_test)
 
    return preproc, X_train, X_test, y_train, y_test

def train(X_train,y_train):
    '''
    train the model
    NOTE: unable to use the optimal parameters to train from param_search.py ..
    because the size model.joblib was too large for streamlit to process. 
    '''
    forest = RandomForestClassifier(n_estimators=10, random_state=1, criterion= 'gini' )
    clf = MultiOutputClassifier(forest, n_jobs = -3)
    clf.fit(X_train,y_train)
    return clf

def test(X_test,y_test):
    '''evaluate the model's performance on test data'''
    y_pred = clf.predict(X_test)
    accuracy = 1 - hamming_loss(y_test,y_pred)
    print(f"accuracy: {accuracy}")
    return accuracy

def train_full(df):
    '''train the model on the full dataset'''
    X = df[df.columns[89:]]
    y = df[df.columns[3:89]]

    std_scaler = StandardScaler()
    X = std_scaler.fit_transform(X)
    pca = PCA(n_components = 46)
    pca.fit(X)
    X = pca.transform(X)
    model = train(X,y)

    return pca, model

def save_model_joblib(model):
    '''saving models'''
    joblib.dump(model, 'model.joblib', compress = 3)
    print("saved model.joblib locally")

def save_preproc(model):
    '''saving prepoc model'''
    joblib.dump(model,'preproc.joblib')
    print("saved preproc.joblib locally")

if __name__ == "__main__":
    df = get_data()
    preproc, X_train, X_test, y_train, y_test = preprocess(df)
    clf = train(X_train,y_train)
    test(X_test,y_test)
    save_model_joblib(clf)
    save_preproc(preproc)
    size_bytes = os.stat('model.joblib',).st_size
    print(f"size_bytes is {size_bytes}")

accuracy: 0.7386731411347742
saved model.joblib locally
saved preproc.joblib locally
size_bytes is 127048666


In [41]:
import warnings
warnings.filterwarnings('ignore')

import joblib
import pandas as pd
from mordred import Calculator, descriptors
from rdkit import Chem
from urllib.request import urlopen
from urllib.parse import quote

from ddi.utils import get_data_filepath

In [42]:
Y_class = pd.read_csv(get_data_filepath('complete_severity_reclassification.csv'),
                      usecols = ['sub_system_severity','Y_cat'])
Y_class = Y_class[(Y_class['Y_cat'] != 86)]

# Mordred calculates over 1300 molecular features of the drugs, but as only 721
# features used for prediction (feature column names are reflected in feat_eng_df)
feat_eng_df = pd.read_csv(get_data_filepath('feature_engineering.csv'), nrows=0)
X_cols = feat_eng_df.columns[1:]


def get_smiles(drug1,drug2):
    '''Converts drug names to smiles structures, returns try again
    error if the drug names cannot be converted by the API
    '''
    smile_list = []
    for drug in [drug1,drug2]:
        try:
            url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(drug) + '/smiles'
            ans = urlopen(url).read().decode('utf8')
            smile_list.append(ans)
        except:
            smile_list.append('Unable to find the drug, please try again.')

    return smile_list


def get_mordred(drug1, drug2):
    '''Calculates the molecular features of the drugs from their
    smile structures and builds them into a dataframe
    '''
    # Drug names are converted into smiles through API
    smile_list = get_smiles(drug1, drug2)

    # Molecular features dataframe is built from the smile list provided
    mols = [Chem.MolFromSmiles(item) for item in smile_list]
    calc = Calculator(descriptors, ignore_3D=True)
    drug_features = calc.pandas(mols)
    return drug_features


def preproc(drug1, drug2):
    '''Extracts features from the drug names into a dataframe, and perform scaling and PCA transformation'''
    drug_features = get_mordred(drug1, drug2)
    # Booleans in the dataframe are converted into numbers 0 and 1
    drug_features.replace({False: 0, True: 1}, inplace=True)
    # Filter out only the necessary 721 columns
    drug_features = drug_features[X_cols]
    # Features are differenced to calculate the similarity of molecular features between the drug pair
    X_test = pd.DataFrame(drug_features.iloc[0] - drug_features.iloc[1]).transpose()
    # Obtaining absolute values as we are only interested in the magnitude of the difference between features
    X_test = X_test.abs()
    # PCA is performed to reduce the dimensionality of the differenced features
    preproc = joblib.load('preproc.joblib')
    X_test = preproc.transform(X_test)

    return X_test


def predict(drug1, drug2, model):
    '''Predicts the target class numbers from the input drug combination'''
    # Features dataframe is built
    X_test = preproc(drug1, drug2)
    # Model is used to predict the target class numbers
    y_pred = model.predict(X_test)
    return y_pred


def classify(drug1, drug2, model):
    '''Converts the predicted class numbers into names of classes predicted'''
    y_pred = predict(drug1, drug2, model)[0]
    # A dictionary is created that returns the predicted sub_system as values
    y_dict = pd.Series(Y_class.sub_system_severity.values, index = Y_class.Y_cat).to_dict()

    # The predicted classes are retrieved and stored into a list'''
    prediction_list = []
    for i, x in enumerate(y_pred):
        if x == 0:
            continue
        prediction_list.append(i)

    # The predicted side effects are retrieved given the predicted categories'''
    side_effect_list = []
    for i in prediction_list:
        side_effect_list.append(y_dict[i])

    return side_effect_list

if __name__ == "__main__":
    model = joblib.load('model.joblib')
    print(classify('Aspirin','Paracetamol', model))


100%|█████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.72it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'preproc.joblib'